In [3]:
import os

In [4]:
os.chdir('../')

## About callbacks

* software development, especially in frameworks like TensorFlow or Keras for building neural networks, "callbacks" are functions that are called at specific points during the training of a model. These callbacks serve various purposes, such as logging information, saving model checkpoints, adjusting learning rates dynamically, or stopping training early based on certain conditions.

* "Prepare callbacks" typically refer to a subset of callbacks that are invoked before the training begins. They allow you to set up the environment or configure certain aspects of the training process. Here are some common tasks that "prepare callbacks" might handle

1. Data Preprocessing: Callbacks can be used to preprocess the data before feeding it into the model. This might involve normalization, resizing, or any other preprocessing steps necessary to prepare the data for training.

2. Model Initialization: You might want to initialize the model or its parameters in a specific way before training begins. Callbacks can be used to perform such initialization tasks.

3. Setting up Logging: Callbacks can set up logging mechanisms to track various metrics during training, such as loss, accuracy, or any other custom metrics.

4. Preparing Learning Rate Schedules: If you plan to use learning rate schedules during training (e.g., decreasing the learning rate over time), you can set up these schedules using callbacks before training starts.

5. Configuring Early Stopping Criteria: Callbacks can be used to configure early stopping criteria based on validation metrics. This allows you to stop training early if the model's performance on a validation set doesn't improve after a certain number of epochs.

6. Setting up Checkpointing: Callbacks can configure the model to save checkpoints at regular intervals during training. These checkpoints can be used to resume training from a specific point or to evaluate the model's performance on unseen data.

By using "prepare callbacks," you can customize the training process and ensure that everything is set up correctly before the actual training begins. This modular approach enhances the flexibility and extensibility of the training process, allowing you to tailor it to your specific requirements.







When ever we are creating a model, it will save the metadata on the callbacks folders. When you lauch the tensorboard , you can see the Training graph, how it performing, what is the error ? we can even find the best model out of many epochs. If your model's performance not improving after some point when doing traing we can stop the traing at that point. 

In [5]:
%pwd

'c:\\Users\\karth\\Documents\\EtE_prj\\Chicken-Disease-Classification'

In [6]:
cd "Chicken-Disease-Classification"

[WinError 2] The system cannot find the file specified: 'Chicken-Disease-Classification'
c:\Users\karth\Documents\EtE_prj\Chicken-Disease-Classification\research


c:\Users\karth\.conda\envs\chicken\Lib\site-packages\IPython\core\magics\osm.py:393: UserWarning: This is now an optional IPython functionality, using bookmarks requires you to install the `pickleshare` library.
  bkms = self.shell.db.get('bookmarks', {})


In [30]:
from dataclasses import dataclass
from pathlib import Path

#creating entity for call back folder configuration
@dataclass(frozen=True)
class PrepareCallbacksConfig:
    root_dir: Path
    tensorboard_root_log_dir: Path
    checkpoint_model_filepath: Path

In [31]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories

In [32]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])


    
    def get_prepare_callback_config(self) -> PrepareCallbacksConfig:
        config = self.config.prepare_callbacks
        model_ckpt_dir = os.path.dirname(config.checkpoint_model_filepath)
        create_directories([
            Path(model_ckpt_dir),
            Path(config.tensorboard_root_log_dir)
        ])

        prepare_callback_config = PrepareCallbacksConfig(
            root_dir=Path(config.root_dir),
            tensorboard_root_log_dir=Path(config.tensorboard_root_log_dir),
            checkpoint_model_filepath=Path(config.checkpoint_model_filepath)
        )

        return prepare_callback_config

In [33]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time

In [34]:
class PrepareCallback:
    def __init__(self, config: PrepareCallbacksConfig):
        self.config = config


    
    @property
    def _create_tb_callbacks(self): # When running code at some time, it will create a folder with that timestamp and stores the logs 
        timestamp = time.strftime("%Y-%m-%d-%H-%M-%S") #Getting timestamp
        tb_running_log_dir = os.path.join(
            self.config.tensorboard_root_log_dir,
            f"tb_logs_at_{timestamp}",# creating a folder with name which contains timestamp
        )
        return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)
    

    @property
    def _create_ckpt_callbacks(self): # It creates a folder and saves check points for a model 
        return tf.keras.callbacks.ModelCheckpoint(
            filepath=str(self.config.checkpoint_model_filepath),
            save_best_only=True
        )

# If you wanna get out of checkpoints you can get it from here 
    def get_tb_ckpt_callbacks(self):
        return [
            self._create_tb_callbacks,
            self._create_ckpt_callbacks
        ]

In [35]:
try:
    config = ConfigurationManager()
    prepare_callbacks_config = config.get_prepare_callback_config()
    prepare_callbacks = PrepareCallback(config=prepare_callbacks_config)
    callback_list = prepare_callbacks.get_tb_ckpt_callbacks()
    
except Exception as e:
    raise e

[2023-12-03 12:37:32,261:INFO:common:yaml file: config\config.yaml loaded successfully]
[2023-12-03 12:37:32,269:INFO:common:yaml file: params.yaml loaded successfully]
[2023-12-03 12:37:32,272:INFO:common:created directory at: artifacts]
[2023-12-03 12:37:32,274:INFO:common:created directory at: artifacts\prepare_callbacks\checkpoint_dir]
[2023-12-03 12:37:32,276:INFO:common:created directory at: artifacts\prepare_callbacks\tensorboard_log_dir]


In [36]:
import time
timestamp = time.strftime("%Y-%m-%d-%H-%M-%S") #Getting timestamp
f"tb_logs_at_{timestamp}" # Creating folder with tb_logs_at + Timestamp attached

'tb_logs_at_2023-12-03-12-57-13'